# Back-Translation

In [1]:
!pip install -q transformers sentencepiece datasets accelerate evaluate sacrebleu

In [2]:
import os
import numpy as np
import torch
from torch.utils.data import Dataset
from datasets import load_dataset
import evaluate
from transformers import (
    AutoTokenizer,
    MBart50TokenizerFast,
    AutoModelForSeq2SeqLM,
    DataCollatorForSeq2Seq,
    Seq2SeqTrainingArguments,
    Seq2SeqTrainer
)

2024-05-14 18:54:33.970162: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-05-14 18:54:33.970277: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-05-14 18:54:34.120475: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered


In [3]:
import wandb
wandb.login(key="41546a42a22875e1707e92f43d24a95281b54784")

wandb: W&B API key is configured. Use `wandb login --relogin` to force relogin
wandb: WARNING If you're specifying your api key in code, ensure this code is not shared publicly.
wandb: WARNING Consider setting the WANDB_API_KEY environment variable, or running `wandb login` from the command line.
wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc


True

In [4]:
path = '/kaggle/input/mbart50_vi-en/pytorch/ver1/1/checkpoint-16500'

In [5]:
tokenizer = AutoTokenizer.from_pretrained(path)
model = AutoModelForSeq2SeqLM.from_pretrained(path)

**Synthetic Data**

Vi Monolingual Data: https://drive.google.com/drive/folders/1brbflvjGcF7GqDQFxafUbGcbf68jCB0g?usp=sharing

In [6]:
with open('/kaggle/input/phomt-data/test.vi', 'r') as f:
    vi_phomt = f.readlines()

In [7]:
vi_phomt[:5]

['Anh Albert Barnett và chị Susan Barnett , thuộc hội thánh West ở Tuscaloosa , Alabama\n',
 'Ngày 11 và 12-1-2020 , những cơn bão lớn đã quét qua và phá huỷ nhiều vùng ở miền nam và miền trung Hoa Kỳ .\n',
 'Những trận mưa to và gió lớn trong suốt hai ngày cùng với nhiều cơn lốc xoáy đã gây thiệt hại nặng nề cho nhiều bang .\n',
 'Đáng buồn là anh Albert Barnett 85 tuổi , và vợ anh là chị Susan Barnett 75 tuổi đã thiệt mạng do một cơn lốc xoáy quét qua nhà họ .\n',
 'Chi nhánh Hoa Kỳ cũng cho biết có ít nhất bốn căn nhà của anh em chúng tôi và hai Phòng Nước Trời bị hư hại nhẹ .\n']

In [8]:
len(vi_phomt)

19151

In [9]:
vi_phomt[:10]

['Anh Albert Barnett và chị Susan Barnett , thuộc hội thánh West ở Tuscaloosa , Alabama\n',
 'Ngày 11 và 12-1-2020 , những cơn bão lớn đã quét qua và phá huỷ nhiều vùng ở miền nam và miền trung Hoa Kỳ .\n',
 'Những trận mưa to và gió lớn trong suốt hai ngày cùng với nhiều cơn lốc xoáy đã gây thiệt hại nặng nề cho nhiều bang .\n',
 'Đáng buồn là anh Albert Barnett 85 tuổi , và vợ anh là chị Susan Barnett 75 tuổi đã thiệt mạng do một cơn lốc xoáy quét qua nhà họ .\n',
 'Chi nhánh Hoa Kỳ cũng cho biết có ít nhất bốn căn nhà của anh em chúng tôi và hai Phòng Nước Trời bị hư hại nhẹ .\n',
 'Ngoài ra , những cơn bão cũng gây hư hại lớn cho cơ sở kinh doanh của một anh em .\n',
 'Các trưởng lão địa phương và giám thị xung quanh đang giúp đỡ và cung cấp về vật chất và tinh thần cho các anh chị bị ảnh hưởng trong thảm hoạ này .\n',
 'Chúng ta tin chắc rằng Cha trên trời , Đức Giê-hô-va , đang an ủi những anh chị em của chúng ta trong cảnh đau buồn .\n',
 'Các cơ quan và viên chức chính phủ quốc

In [10]:
def inference(
    text,
    tokenizer,
    model,
    device="cuda",
    max_length=75,
    beam_size=5
    ):
    inputs = tokenizer(
        text,
        padding="max_length",
        truncation=True,
        max_length=max_length,
        return_tensors="pt"
        )
    input_ids = inputs.input_ids.to(device)
    attention_mask = inputs.attention_mask.to(device)
    model.to(device)

    outputs = model.generate(
        input_ids,
        attention_mask=attention_mask,
        max_length=max_length,
        early_stopping=True,
        num_beams=beam_size,
        length_penalty=2.0
    )

    output_str = tokenizer.batch_decode(outputs, skip_special_tokens=True)

    return output_str[0]

en_phomt_preds = []
for sent in vi_phomt:
    en_sent = inference(sent, tokenizer, model)
    en_phomt_preds.append(en_sent)

In [11]:
en_phomt_preds[:10]

['Albert Barnett and his sister Susan Barnett, from the West Church in Tuscaloosa, Alabama.',
 'On November and December, 2020, major storms came through and subverted parts of the south and central United States.',
 'The heavy rains and the heavy winds, over two days, along with the hurricanes, caused extensive damage to many states.',
 'Sadly, his brother Albert Barnett was 85, and his wife, Susan Barnett, was 75 when a tornado struck their home.',
 'The U.S. branch also said that there were at least four of our brothers &apos; houses and two of our Sun Halls that were slightly damaged.',
 'Also, the storms do a huge amount of damage to a brother &apos;s business.',
 'Local elders and local supervisors are helping and providing material and spiritual support to the brothers and sisters affected by this disaster.',
 'We know for certain that the Lord God, Jesus Christ, was there to comfort our brothers and sisters in their distress.',
 'International organizations and government offic

**En-VI**

In [12]:
class NMTDataset(Dataset):
    def __init__(self, cfg, src_augs=[], tgt_augs=[], data_type="train"):
        super().__init__()
        self.cfg = cfg

        self.src_texts, self.tgt_texts = self.read_data(data_type)
        if data_type == 'train':
            self.src_texts.extend(src_augs)
            self.tgt_texts.extend(tgt_augs)

        self.src_input_ids = self.texts_to_sequences(self.src_texts)
        self.labels = self.texts_to_sequences(self.tgt_texts)

    def read_data(self, data_type):
        data = load_dataset(
            "mt_eng_vietnamese",
            "iwslt2015-en-vi",
            split=data_type
        )
        src_texts = [sample["translation"][self.cfg.src_lang] for sample in data]
        tgt_texts = [sample["translation"][self.cfg.tgt_lang] for sample in data]
        return src_texts, tgt_texts

    def texts_to_sequences(self, texts):
        data_inputs = self.cfg.tokenizer(
            texts,
            padding='max_length',
            truncation=True,
            max_length=self.cfg.max_len,
            return_tensors='pt'
        )
        return data_inputs.input_ids

    def __getitem__(self, idx):
        return {
            "input_ids": self.src_input_ids[idx],
            "labels": self.labels[idx]
        }

    def __len__(self):
        return np.shape(self.src_input_ids)[0]

In [13]:
class BaseConfig:
    """ base Encoder Decoder config """

    def __init__(self, **kwargs):
        for k, v in kwargs.items():
            setattr(self, k, v)

class NMTConfig(BaseConfig):
    # Data
    src_lang = 'en'
    tgt_lang = 'vi'
    max_len = 75
    add_special_tokens = True

    # Model
    model_name = "facebook/mbart-large-50-many-to-many-mmt"

    # Training
    device = torch.device('cuda') if torch.cuda.is_available() else torch.device('cpu')
    learning_rate = 5e-5
    train_batch_size = 16
    eval_batch_size = 16
    num_train_epochs = 3
    save_total_limit = 1
    ckpt_dir = f'./mbart50-{src_lang}-{tgt_lang}-backtranslation'
    eval_steps = 1000

    # Inference
    beam_size = 5

cfg = NMTConfig()

In [14]:
# tokenizer = MBart50TokenizerFast.from_pretrained(cfg.model_name, src_lang="en_XX",tgt_lang = "vi_VN")
cfg.tokenizer = MBart50TokenizerFast.from_pretrained(cfg.model_name)
model = AutoModelForSeq2SeqLM.from_pretrained(cfg.model_name)

tokenizer_config.json:   0%|          | 0.00/529 [00:00<?, ?B/s]

sentencepiece.bpe.model:   0%|          | 0.00/5.07M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/649 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/1.43k [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/2.44G [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/261 [00:00<?, ?B/s]

In [15]:
train_dataset = NMTDataset(cfg, en_phomt_preds, vi_phomt, data_type="train")
valid_dataset = NMTDataset(cfg, data_type="validation")
test_dataset = NMTDataset(cfg, data_type="test")

Generating train split:   0%|          | 0/133318 [00:00<?, ? examples/s]

Generating validation split:   0%|          | 0/1269 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/1269 [00:00<?, ? examples/s]

In [16]:
len(train_dataset)

152469

In [17]:
metric = evaluate.load("sacrebleu")

def postprocess_text(preds, labels):
    preds = [pred.strip() for pred in preds]
    labels = [[label.strip()] for label in labels]

    return preds, labels

def compute_metrics(eval_preds):
    preds, labels = eval_preds
    if isinstance(preds, tuple):
        preds = preds[0]

    preds= np.where(preds != -100, preds, cfg.tokenizer.pad_token_id)
    decoded_preds = cfg.tokenizer.batch_decode(preds, skip_special_tokens=True, clean_up_tokenization_spaces=True)

    labels= np.where(labels != -100, labels, cfg.tokenizer.pad_token_id)
    decoded_labels = cfg.tokenizer.batch_decode(labels, skip_special_tokens=True, clean_up_tokenization_spaces=True)

    decoded_preds, decoded_labels = postprocess_text(decoded_preds, decoded_labels)

    result = metric.compute(predictions=decoded_preds, references=decoded_labels)
    result = {"bleu": result["score"]}

    prediction_lens = [np.count_nonzero(pred != cfg.tokenizer.pad_token_id) for pred in preds]
    result["gen_len"] = np.mean(prediction_lens)
    result = {k: round(v, 4) for k, v in result.items()}

    return result

In [18]:
training_args = Seq2SeqTrainingArguments(
    predict_with_generate=True,
    evaluation_strategy="steps",
    eval_steps=cfg.eval_steps,
    output_dir=cfg.ckpt_dir,
    per_device_train_batch_size=cfg.train_batch_size,
    per_device_eval_batch_size=cfg.eval_batch_size,
    learning_rate=cfg.learning_rate,
    save_total_limit=cfg.save_total_limit,
    num_train_epochs=cfg.num_train_epochs,
    load_best_model_at_end=False,
)

data_collator = DataCollatorForSeq2Seq(
    cfg.tokenizer,
    model=model
)

trainer = Seq2SeqTrainer(
    model,
    training_args,
    train_dataset=train_dataset,
    eval_dataset=valid_dataset,
    data_collator=data_collator,
    tokenizer=cfg.tokenizer,
    compute_metrics=compute_metrics
)

/opt/conda/lib/python3.10/site-packages/accelerate/accelerator.py:436: FutureWarning: Passing the following arguments to `Accelerator` is deprecated and will be removed in version 1.0 of Accelerate: dict_keys(['dispatch_batches', 'split_batches', 'even_batches', 'use_seedable_sampler']). Please pass an `accelerate.DataLoaderConfiguration` instead: 
dataloader_config = DataLoaderConfiguration(dispatch_batches=None, split_batches=False, even_batches=True, use_seedable_sampler=True)
  warnings.warn(


In [19]:
trainer.train()

wandb: Currently logged in as: namanh2k2av (nlnamanh2002). Use `wandb login --relogin` to force relogin
wandb: wandb version 0.17.0 is available!  To upgrade, please run:
wandb:  $ pip install wandb --upgrade
wandb: Tracking run with wandb version 0.16.6
wandb: Run data is saved locally in /kaggle/working/wandb/run-20240514_210104-8kpcv5pa
wandb: Run `wandb offline` to turn off syncing.
wandb: Syncing run spring-dust-26
wandb: ⭐️ View project at https://wandb.ai/nlnamanh2002/huggingface
wandb: 🚀 View run at https://wandb.ai/nlnamanh2002/huggingface/runs/8kpcv5pa


Step,Training Loss,Validation Loss,Bleu,Gen Len
1000,0.499800,0.589652,32.616100,32.801400
2000,0.491000,0.581128,32.112300,33.331000
3000,0.490800,0.572532,32.716400,32.820300
4000,0.483500,0.565749,32.873700,33.157600
5000,0.486200,0.558821,33.693000,32.580000
6000,0.475700,0.552827,33.602800,32.565800
7000,0.468200,0.551341,33.737700,32.803000
8000,0.471000,0.544808,33.737600,32.649300
9000,0.456500,0.542040,33.717600,32.268700
10000,0.375100,0.555766,33.578200,32.890500


Some non-default generation parameters are set in the model config. These should go into a GenerationConfig file (https://huggingface.co/docs/transformers/generation_strategies#save-a-custom-decoding-strategy-with-your-model) instead. This warning will be raised to an exception in v4.41.
Non-default generation parameters: {'max_length': 200, 'early_stopping': True, 'num_beams': 5, 'forced_eos_token_id': 2}
Some non-default generation parameters are set in the model config. These should go into a GenerationConfig file (https://huggingface.co/docs/transformers/generation_strategies#save-a-custom-decoding-strategy-with-your-model) instead. This warning will be raised to an exception in v4.41.
Non-default generation parameters: {'max_length': 200, 'early_stopping': True, 'num_beams': 5, 'forced_eos_token_id': 2}
Some non-default generation parameters are set in the model config. These should go into a GenerationConfig file (https://huggingface.co/docs/transformers/generation_strategies#sav

TrainOutput(global_step=28590, training_loss=0.38018773443509585, metrics={'train_runtime': 29818.5437, 'train_samples_per_second': 15.34, 'train_steps_per_second': 0.959, 'total_flos': 7.26020746407936e+16, 'train_loss': 0.38018773443509585, 'epoch': 3.0})

In [20]:
trainer.evaluate(test_dataset)

{'eval_loss': 0.556479275226593,
 'eval_bleu': 34.4047,
 'eval_gen_len': 32.773,
 'eval_runtime': 204.6862,
 'eval_samples_per_second': 6.2,
 'eval_steps_per_second': 0.391,
 'epoch': 3.0}

In [21]:
trainer.save_model()

Some non-default generation parameters are set in the model config. These should go into a GenerationConfig file (https://huggingface.co/docs/transformers/generation_strategies#save-a-custom-decoding-strategy-with-your-model) instead. This warning will be raised to an exception in v4.41.
Non-default generation parameters: {'max_length': 200, 'early_stopping': True, 'num_beams': 5, 'forced_eos_token_id': 2}
